In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive/')

# Change directory
%cd /content/drive/My Drive/CISC_CMPE351

Mounted at /content/drive/
/content/drive/My Drive/CISC_CMPE351


In [ ]:
# Imports
import pandas as pd

# Numpy for calculations
import numpy as np
from numpy import mean
from numpy import std

# Sci-kit Learn
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV

# LightGBM
import lightgbm as lgb
from lightgbm import LGBMRegressor
from lightgbm import plot_importance

# For plotting
import matplotlib.pyplot as plt
from matplotlib import pyplot

In [ ]:
# Questions to be Answered
# Which sub-reddit/redditor should be followed based on risk and expected return (sharpe ratio)?
# How much price movement can be expected from the stock?
# How long will the trade be worthwhile?

# Load file into pandas data frame
myDataFrame = pd.read_csv('1dayWindow_original_no_duplicates_date_tick.csv')

# Drop rows with NaN
myDataFrame.dropna(inplace=True)

# Display
myDataFrame.head()

,date,ticker,volume_20_X,adjclose_20_X,close_20_X,low_20_X,high_20_X,open_20_X,volume_19_X,adjclose_19_X,close_19_X,low_19_X,high_19_X,open_19_X,volume_18_X,adjclose_18_X,close_18_X,low_18_X,high_18_X,open_18_X,volume_17_X,adjclose_17_X,close_17_X,low_17_X,high_17_X,open_17_X,volume_16_X,adjclose_16_X,close_16_X,low_16_X,high_16_X,open_16_X,volume_15_X,adjclose_15_X,close_15_X,low_15_X,high_15_X,open_15_X,volume_14_X,adjclose_14_X,...,low_16_Y,close_16_Y,adjclose_16_Y,volume_16_Y,open_17_Y,high_17_Y,low_17_Y,close_17_Y,adjclose_17_Y,volume_17_Y,open_18_Y,high_18_Y,low_18_Y,close_18_Y,adjclose_18_Y,volume_18_Y,open_19_Y,high_19_Y,low_19_Y,close_19_Y,adjclose_19_Y,volume_19_Y,open_20_Y,high_20_Y,low_20_Y,close_20_Y,adjclose_20_Y,volume_20_Y,open_21_Y,high_21_Y,low_21_Y,close_21_Y,adjclose_21_Y,volume_21_Y,post_type,sum_comments,sum_score,sum_awards,mean_sentiment,num_posts
0,2020-02-08,FNKO,1787600.0,15.790000,15.790000,15.450000,16.480000,16.480000,1377400.0,16.510000,16.510000,16.375000,16.790001,16.719999,1025600.0,16.850000,16.850000,16.750000,17.370001,17.170000,821400.0,17.570000,17.570000,17.056999,17.760000,17.260000,1009000.0,17.160000,17.160000,16.709999,17.270000,16.900000,905000.0,16.895000,16.895000,16.510000,17.260000,16.910000,897000.0,16.879999,...,7.280000,7.390000,7.390000,778500.0,7.480000,7.600000,7.340000,7.530000,7.530000,730800.0,7.360000,7.450000,7.080000,7.240000,7.240000,1372600.0,6.710000,7.570000,6.510000,6.920000,6.920000,1531200.0,6.290000,6.685000,6.060000,6.300000,6.300000,1019000.0,6.480000,6.560000,5.900000,6.210000,6.210000,881000.0,0,0,1,0.0,3.440400,1
1,2020-02-08,AAPL,108872000.0,73.448982,74.597504,74.370003,75.224998,74.959999,118387200.0,73.796051,74.949997,73.187500,74.989998,73.447502,146322800.0,73.212669,74.357498,74.125000,75.144997,74.287498,135480400.0,73.931435,75.087502,73.797501,75.150002,74.059998,100805600.0,72.282227,73.412498,72.379997,73.419998,72.482498,144114400.0,71.757919,72.879997,71.305000,73.172501,72.364998,146266000.0,71.334534,...,71.449997,72.330002,71.385406,319475600.0,74.110001,75.849998,73.282501,75.684998,74.696587,219178400.0,73.879997,74.887497,72.852501,73.230003,72.273659,187572800.0,70.500000,72.705002,70.307503,72.257500,71.313866,226176800.0,65.937500,69.522499,65.750000,66.542503,65.673492,286744800.0,69.285004,71.610001,67.342499,71.334999,70.403404,285290000.0,0,10,187,0.0,0.051900,19
2,2020-02-08,JAN,15700.0,3.130000,3.130000,2.970000,3.180000,2.970000,19100.0,3.030000,3.030000,3.010000,3.190000,3.170000,36300.0,3.100000,3.100000,2.960000,3.200000,3.050000,102800.0,3.030000,3.030000,2.820000,3.390000,2.880000,29600.0,2.960000,2.960000,2.810000,2.960000,2.940000,13800.0,2.870000,2.870000,2.800000,2.940000,2.940000,27000.0,2.930000,...,3.560000,3.700000,3.700000,16000.0,3.610000,4.040000,3.600000,3.900000,3.900000,53400.0,3.900000,3.930000,3.780000,3.780000,3.780000,15200.0,3.720000,3.840000,3.600000,3.800000,3.800000,25700.0,3.600000,3.730000,3.460000,3.530000,3.530000,28700.0,3.590000,3.690000,3.300000,3.680000,3.680000,33400.0,0,0,1,0.0,2.585900,1
3,2020-02-08,ATO,875500.0,108.463562,111.699997,110.430000,112.019997,112.000000,822900.0,107.745026,110.959999,110.889999,111.680000,110.959999,724200.0,107.890678,111.110001,110.059998,111.320000,110.059998,906900.0,107.181824,110.379997,109.419998,112.040001,112.040001,637900.0,108.618942,111.860001,110.379997,111.879997,110.440002,500700.0,107.269211,110.470001,109.750000,110.480003,109.949997,648600.0,106.939064,...,105.690002,106.540001,103.954407,1146700.0,107.940002,113.150002,107.940002,112.900002,110.160065,1106300.0,111.019997,111.900002,110.190002,111.129997,108.433014,1204600.0,107.599998,111.669998,106.500000,111.250000,108.550110,1557800.0,107.110001,107.470001,103.330002,105.129997,102.578629,1519800.0,106.860001,107.309998,101.160004,105.430000,102.871353,1418500.0,0,4,15,0.0,-0.227048,7
4,2020-02-08,TSLA,89410500.0,93.811996,93.811996,90.671997,94.325996,92.279999,50665000.0,90.307999,

In [ ]:
# Percentage Conversion
for i in range(2, 22):
    myDataFrame['adjclose_{}_Y'.format(i)] = (myDataFrame['adjclose_{}_Y'.format(i)] - myDataFrame['adjclose_1_X'])/myDataFrame['adjclose_1_X']
    
myDataFrame.head(10)

,date,ticker,volume_20_X,adjclose_20_X,close_20_X,low_20_X,high_20_X,open_20_X,volume_19_X,adjclose_19_X,close_19_X,low_19_X,high_19_X,open_19_X,volume_18_X,adjclose_18_X,close_18_X,low_18_X,high_18_X,open_18_X,volume_17_X,adjclose_17_X,close_17_X,low_17_X,high_17_X,open_17_X,volume_16_X,adjclose_16_X,close_16_X,low_16_X,high_16_X,open_16_X,volume_15_X,adjclose_15_X,close_15_X,low_15_X,high_15_X,open_15_X,volume_14_X,adjclose_14_X,...,low_16_Y,close_16_Y,adjclose_16_Y,volume_16_Y,open_17_Y,high_17_Y,low_17_Y,close_17_Y,adjclose_17_Y,volume_17_Y,open_18_Y,high_18_Y,low_18_Y,close_18_Y,adjclose_18_Y,volume_18_Y,open_19_Y,high_19_Y,low_19_Y,close_19_Y,adjclose_19_Y,volume_19_Y,open_20_Y,high_20_Y,low_20_Y,close_20_Y,adjclose_20_Y,volume_20_Y,open_21_Y,high_21_Y,low_21_Y,close_21_Y,adjclose_21_Y,volume_21_Y,post_type,sum_comments,sum_score,sum_awards,mean_sentiment,num_posts
0,2020-02-08,FNKO,1787600.0,15.790000,15.790000,15.450000,16.480000,16.480000,1377400.0,16.510000,16.510000,16.375000,16.790001,16.719999,1025600.0,16.850000,16.850000,16.750000,17.370001,17.170000,821400.0,17.570000,17.570000,17.056999,17.760000,17.260000,1009000.0,17.160000,17.160000,16.709999,17.270000,16.900000,905000.0,16.895000,16.895000,16.510000,17.260000,16.910000,897000.0,16.879999,...,7.280000,7.390000,-0.514455,778500.0,7.480000,7.600000,7.340000,7.530000,-0.505256,730800.0,7.360000,7.450000,7.080000,7.240000,-0.524310,1372600.0,6.710000,7.570000,6.510000,6.920000,-0.545335,1531200.0,6.290000,6.685000,6.060000,6.300000,-0.586071,1019000.0,6.480000,6.560000,5.900000,6.210000,-0.591984,881000.0,0,0,1,0.0,3.440400,1
1,2020-02-08,AAPL,108872000.0,73.448982,74.597504,74.370003,75.224998,74.959999,118387200.0,73.796051,74.949997,73.187500,74.989998,73.447502,146322800.0,73.212669,74.357498,74.125000,75.144997,74.287498,135480400.0,73.931435,75.087502,73.797501,75.150002,74.059998,100805600.0,72.282227,73.412498,72.379997,73.419998,72.482498,144114400.0,71.757919,72.879997,71.305000,73.172501,72.364998,146266000.0,71.334534,...,71.449997,72.330002,0.086493,319475600.0,74.110001,75.849998,73.282501,75.684998,0.136889,219178400.0,73.879997,74.887497,72.852501,73.230003,0.100012,187572800.0,70.500000,72.705002,70.307503,72.257500,0.085404,226176800.0,65.937500,69.522499,65.750000,66.542503,-0.000443,286744800.0,69.285004,71.610001,67.342499,71.334999,0.071546,285290000.0,0,10,187,0.0,0.051900,19
2,2020-02-08,JAN,15700.0,3.130000,3.130000,2.970000,3.180000,2.970000,19100.0,3.030000,3.030000,3.010000,3.190000,3.170000,36300.0,3.100000,3.100000,2.960000,3.200000,3.050000,102800.0,3.030000,3.030000,2.820000,3.390000,2.880000,29600.0,2.960000,2.960000,2.810000,2.960000,2.940000,13800.0,2.870000,2.870000,2.800000,2.940000,2.940000,27000.0,2.930000,...,3.560000,3.700000,0.330935,16000.0,3.610000,4.040000,3.600000,3.900000,0.402878,53400.0,3.900000,3.930000,3.780000,3.780000,0.359712,15200.0,3.720000,3.840000,3.600000,3.800000,0.366906,25700.0,3.600000,3.730000,3.460000,3.530000,0.269784,28700.0,3.590000,3.690000,3.300000,3.680000,0.323741,33400.0,0,0,1,0.0,2.585900,1
3,2020-02-08,ATO,875500.0,108.463562,111.699997,110.430000,112.019997,112.000000,822900.0,107.745026,110.959999,110.889999,111.680000,110.959999,724200.0,107.890678,111.110001,110.059998,111.320000,110.059998,906900.0,107.181824,110.379997,109.419998,112.040001,112.040001,637900.0,108.618942,111.860001,110.379997,111.879997,110.440002,500700.0,107.269211,110.470001,109.750000,110.480003,109.949997,648600.0,106.939064,...,105.690002,106.540001,0.007020,1146700.0,107.940002,113.150002,107.940002,112.900002,0.067135,1106300.0,111.019997,111.900002,110.190002,111.129997,0.050405,1204600.0,107.599998,111.669998,106.500000,111.250000,0.051539,1557800.0,107.110001,107.470001,103.330002,105.129997,-0.006307,1519800.0,106.860001,107.309998,101.160004,105.430000,-0.003472,1418500.0,0,4,15,0.0,-0.227048,7
4,2020-02-08,TSLA,89410500.0,93.811996,93.811996,90.671997,94.325996,92.279999,50665000.0,90.307999,90.307999

In [ ]:
# Column Reference 
def columnReference(DataSet):
    index = 0
    for column in DataSet.columns:
        print(index, ":", column)
        index += 1
columnReference(myDataFrame)

0 : date
1 : ticker
2 : volume_20_X
3 : adjclose_20_X
4 : close_20_X
5 : low_20_X
6 : high_20_X
7 : open_20_X
8 : volume_19_X
9 : adjclose_19_X
10 : close_19_X
11 : low_19_X
12 : high_19_X
13 : open_19_X
14 : volume_18_X
15 : adjclose_18_X
16 : close_18_X
17 : low_18_X
18 : high_18_X
19 : open_18_X
20 : volume_17_X
21 : adjclose_17_X
22 : close_17_X
23 : low_17_X
24 : high_17_X
25 : open_17_X
26 : volume_16_X
27 : adjclose_16_X
28 : close_16_X
29 : low_16_X
30 : high_16_X
31 : open_16_X
32 : volume_15_X
33 : adjclose_15_X
34 : close_15_X
35 : low_15_X
36 : high_15_X
37 : open_15_X
38 : volume_14_X
39 : adjclose_14_X
40 : close_14_X
41 : low_14_X
42 : high_14_X
43 : open_14_X
44 : volume_13_X
45 : adjclose_13_X
46 : close_13_X
47 : low_13_X
48 : high_13_X
49 : open_13_X
50 : volume_12_X
51 : adjclose_12_X
52 : close_12_X
53 : low_12_X
54 : high_12_X
55 : open_12_X
56 : volume_11_X
57 : adjclose_11_X
58 : close_11_X
59 : low_11_X
60 : high_11_X
61 : open_11_X
62 : volume_10_X
63 : adjclo

In [ ]:
# Source: https://machinelearningmastery.com/gradient-boosting-with-scikit-learn-xgboost-lightgbm-and-catboost/

# Get 80% of the data by date
myDataFrame['date'] = pd.to_datetime(myDataFrame['date'])
myDataFrame = myDataFrame.sort_values('date', ascending = False)
myDataFrame = myDataFrame.reset_index(drop=True)
dates = pd.to_datetime(myDataFrame['date'])
myDataFrame = myDataFrame.drop(columns = ['date', 'ticker'])
(dates <= dates.quantile(.8)).idxmax()

# Most recent data for testing
test =  myDataFrame.head(49869)

# Rest of the data is training
train = myDataFrame.tail(-49869)

# Split the data into train and test
X_train = train.iloc[:, list(range(0,120))+list(range(240,246))] 
y_train = train.iloc[:, list(range(124, 244, 6))] 
X_test = test.iloc[:, list(range(0,120))+list(range(240,246))]
y_test = test.iloc[:, list(range(124, 244, 6))]

# Get training column indices
columnReference(X_train)

# Get testing column indices
columnReference(y_train)

0 : volume_20_X
1 : adjclose_20_X
2 : close_20_X
3 : low_20_X
4 : high_20_X
5 : open_20_X
6 : volume_19_X
7 : adjclose_19_X
8 : close_19_X
9 : low_19_X
10 : high_19_X
11 : open_19_X
12 : volume_18_X
13 : adjclose_18_X
14 : close_18_X
15 : low_18_X
16 : high_18_X
17 : open_18_X
18 : volume_17_X
19 : adjclose_17_X
20 : close_17_X
21 : low_17_X
22 : high_17_X
23 : open_17_X
24 : volume_16_X
25 : adjclose_16_X
26 : close_16_X
27 : low_16_X
28 : high_16_X
29 : open_16_X
30 : volume_15_X
31 : adjclose_15_X
32 : close_15_X
33 : low_15_X
34 : high_15_X
35 : open_15_X
36 : volume_14_X
37 : adjclose_14_X
38 : close_14_X
39 : low_14_X
40 : high_14_X
41 : open_14_X
42 : volume_13_X
43 : adjclose_13_X
44 : close_13_X
45 : low_13_X
46 : high_13_X
47 : open_13_X
48 : volume_12_X
49 : adjclose_12_X
50 : close_12_X
51 : low_12_X
52 : high_12_X
53 : open_12_X
54 : volume_11_X
55 : adjclose_11_X
56 : close_11_X
57 : low_11_X
58 : high_11_X
59 : open_11_X
60 : volume_10_X
61 : adjclose_10_X
62 : close_10_

In [ ]:
# Initial Model
myModel = LGBMRegressor(max_depth=12,n_estimators=1000,learning_rate=0.2,max_bin=100,num_leaves=250)

# Wrap Model for Multiple Outputs
wrapper = MultiOutputRegressor(myModel).fit(X_train, y_train)

# Wrapper Prediction
ypred = wrapper.predict(X_test)

In [ ]:
print(ypred)

[[ 1.55574881e+00  1.09683061e+00  7.43138876e-01 ... -6.82561501e+00
  -1.75690515e-01  6.39609940e-01]
 [ 1.92609156e-01 -7.65145923e+01  4.15876162e+01 ... -1.62407917e+03
  -1.86944884e+03 -7.35370787e+00]
 [ 3.96338244e-01  1.64390992e-01  2.87951163e-01 ... -1.41306014e-01
   1.00990250e-01  8.95049781e-02]
 ...
 [ 2.09099229e-01 -8.17108229e-01 -4.87076952e-01 ... -5.43321311e-01
   3.01522977e-01  3.67212726e-01]
 [-5.19157439e-01  1.17954820e+02  7.32000459e+01 ...  1.50703967e+01
  -1.26665690e+01  9.31793785e+00]
 [-4.06558753e-02 -9.25726132e-01 -7.85776089e-01 ... -1.31550385e-01
  -2.46521002e-02 -1.93468054e-01]]


In [ ]:
print(y_test)

       adjclose_2_Y  adjclose_3_Y  ...  adjclose_20_Y  adjclose_21_Y
0          0.708303      0.739162  ...       0.322557       0.382807
1          0.215739      0.233391  ...       0.297867       0.393724
2          0.410478      0.432892  ...       0.523900       0.555226
3          0.603815      0.604182  ...       0.540719       0.584006
4         -0.126837     -0.122406  ...      -0.054457      -0.064276
...             ...           ...  ...            ...            ...
49864      0.790811      0.723446  ...       1.439209       1.540258
49865     -0.086950     -0.084650  ...      -0.069302      -0.056193
49866      0.100000      0.079310  ...       0.189655       0.379310
49867      0.819407      0.819407  ...       0.795148       0.754717
49868     -0.182609     -0.165217  ...      -0.019565      -0.021739

[49869 rows x 20 columns]


In [ ]:
np.savetxt(r"1dayWindow_GBM_ypred.csv", ypred, delimiter=",")
np.savetxt(r"1dayWindow_GBM_ytest.csv", ytest, delimiter=",")